In [1]:
import pandas as pd
import numpy as np
from osst.model.osst import OSST
from osst.model.metrics import harrell_c_index, uno_c_index, integrated_brier_score, cumulative_dynamic_auc, compute_ibs_per_sample
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.datasets import get_x_y
from sklearn.preprocessing import MinMaxScaler
import pathlib


In [3]:
# Loading the data into a datdframe by reading an Excel file 
df = pd.read_excel('LOG_S_1.0mm.xlsx')
df

,A_LOG_1.0S_FO_10Percentile,A_LOG_1.0S_FO_90Percentile,A_LOG_1.0S_FO_Energy,A_LOG_1.0S_FO_Entropy,A_LOG_1.0S_FO_IQR,A_LOG_1.0S_FO_Kurtosis,A_LOG_1.0S_FO_Maximum,A_LOG_1.0S_FO_Mean,A_LOG_1.0S_FO_MAD,A_LOG_1.0S_FO_Median,...,E_shape_MajorAxis,E_shape_Maximum2DDiameterColumn,E_shape_Maximum2DDiameterRow,E_shape_Maximum2DDiameterSlice,E_shape_Maximum3DDiameter,E_shape_Sphericity,E_shape_SurfaceArea,E_shape_SurfaceVolumeRatio,Event,Time
0,-112.993087,34.278236,1.497499e+07,1.950718,78.772847,2.963165,125.661385,-36.069763,45.916473,-32.176239,...,56.413789,110.143944,105.702865,111.731201,47.445347,0.295952,22991.047470,0.435628,1,4.734247
1,-91.982840,19.046037,1.456415e+06,1.733513,45.578664,6.254949,191.171707,-23.584620,35.662827,-11.017938,...,31.705678,47.186476,42.071600,48.412302,26.403203,0.447198,5349.092830,0.486225,1,18.345205
2,-24.633341,7.849291,4.955951e+05,0.966132,13.906203,8.346409,32.401775,-6.966859,10.458822,-4.591255,...,44.359603,41.168907,50.109743,56.818000,34.330462,0.417301,7627.193971,0.451720,1,10.586301
3,-27.706317,15.834357,7.741525e+06,1.168238,17.453013,11.841962,111.435387,-4.398507,14.461138,-1.749925,...,62.334907,69.090980,78.631173,78.631173,56.161281,0.567011,17162.509940,0.190129,1,20.712329
4,-195.595828,99.832056,3.896654e+08,2.882933,96.341311,7.102594,483.333252,-28.100289,91.166116,-7.814033,...,58.462737,60.018560,58.283554,62.152361,47.212345,0.607316,12445.462710,0.201418,1,10.356164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,-157.884051,53.579113,1.734358e+08,2.482020,105.277153,4.452953,317.510162,-43.321775,67.189688,-32.063353,...,74.983177,97.512955,89.737914,98.553894,63.842758,0.365050,34728.398210,0.258735,1,4.832877
115,-140.765985,36.945937,1.381109e+08,2.259325,87.466111,4.180886,250.706039,-45.650616,56.254705,-37.681164,...,75.881498,97.264657,89.431930,104.682305,58.167516,0.426495,31576.815380,0.214868,1,5.030137
116,-307.195953,73.882935,6.998296e+07,3.312063,198.950063,4.621181,407.875977,-112.185588,123.633278,-93.440277,...,69.000674,71.245140,67.843479,72.123861,53.321172,0.466786,19873.790060,0.236542,0,9.271233
117,-282.845697,62.443905,1.537041e+08,3.078824,164.342980,4.190256,342.496735,-90.148083,107.321054,-64.564678,...,75.177316,67.758561,76.448640,76.655989,57.622941,0.550229,19165.348580,0.188214,0,7.726027


In [4]:
df_no_target = df.drop(columns = ['Event', 'Time'])
df_no_target 

,A_LOG_1.0S_FO_10Percentile,A_LOG_1.0S_FO_90Percentile,A_LOG_1.0S_FO_Energy,A_LOG_1.0S_FO_Entropy,A_LOG_1.0S_FO_IQR,A_LOG_1.0S_FO_Kurtosis,A_LOG_1.0S_FO_Maximum,A_LOG_1.0S_FO_Mean,A_LOG_1.0S_FO_MAD,A_LOG_1.0S_FO_Median,...,E_shape_LeastAxis,E_shape_MinorAxis,E_shape_MajorAxis,E_shape_Maximum2DDiameterColumn,E_shape_Maximum2DDiameterRow,E_shape_Maximum2DDiameterSlice,E_shape_Maximum3DDiameter,E_shape_Sphericity,E_shape_SurfaceArea,E_shape_SurfaceVolumeRatio
0,-112.993087,34.278236,1.497499e+07,1.950718,78.772847,2.963165,125.661385,-36.069763,45.916473,-32.176239,...,35.163649,92.011365,56.413789,110.143944,105.702865,111.731201,47.445347,0.295952,22991.047470,0.435628
1,-91.982840,19.046037,1.456415e+06,1.733513,45.578664,6.254949,191.171707,-23.584620,35.662827,-11.017938,...,21.182971,38.551161,31.705678,47.186476,42.071600,48.412302,26.403203,0.447198,5349.092830,0.486225
2,-24.633341,7.849291,4.955951e+05,0.966132,13.906203,8.346409,32.401775,-6.966859,10.458822,-4.591255,...,22.806329,42.325954,44.359603,41.168907,50.109743,56.818000,34.330462,0.417301,7627.193971,0.451720
3,-27.706317,15.834357,7.741525e+06,1.168238,17.453013,11.841962,111.435387,-4.398507,14.461138,-1.749925,...,39.477585,60.426800,62.334907,69.090980,78.631173,78.631173,56.161281,0.567011,17162.509940,0.190129
4,-195.595828,99.832056,3.896654e+08,2.882933,96.341311,7.102594,483.333252,-28.100289,91.166116,-7.814033,...,39.705787,48.178079,58.462737,60.018560,58.283554,62.152361,47.212345,0.607316,12445.462710,0.201418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,-157.884051,53.579113,1.734358e+08,2.482020,105.277153,4.452953,317.510162,-43.321775,67.189688,-32.063353,...,45.947540,81.455505,74.983177,97.512955,89.737914,98.553894,63.842758,0.365050,34728.398210,0.258735
115,-140.765985,36.945937,1.381109e+08,2.259325,87.466111,4.180886,250.706039,-45.650616,56.254705,-37.681164,...,47.845512,84.820549,75.881498,97.264657,89.431930,104.682305,58.167516,0.426495,31576.815380,0.214868
116,-307.195953,73.882935,6.998296e+07,3.312063,198.950063,4.621181,407.875977,-112.185588,123.633278,-93.440277,...,42.888040,57.837507,69.000674,71.245140,67.843479,72.123861,53.321172,0.466786,19873.790060,0.236542
117,-282.845697,62.443905,1.537041e+08,3.078824,164.342980,4.190256,342.496735,-90.148083,107.321054,-64.564678,...,45.435317,61.565039,75.177316,67.758561,76.448640,76.655989,57.622941,0.550229,19165.348580,0.188214


In [4]:
# I will normalize my dataset

In [5]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply normalization to the entire dataframe
df_normalized = pd.DataFrame(scaler.fit_transform(df_no_target), columns=df_no_target.columns)

# Display the normalized dataframe
df_normalized


,A_LOG_1.0S_FO_10Percentile,A_LOG_1.0S_FO_90Percentile,A_LOG_1.0S_FO_Energy,A_LOG_1.0S_FO_Entropy,A_LOG_1.0S_FO_IQR,A_LOG_1.0S_FO_Kurtosis,A_LOG_1.0S_FO_Maximum,A_LOG_1.0S_FO_Mean,A_LOG_1.0S_FO_MAD,A_LOG_1.0S_FO_Median,...,E_shape_LeastAxis,E_shape_MinorAxis,E_shape_MajorAxis,E_shape_Maximum2DDiameterColumn,E_shape_Maximum2DDiameterRow,E_shape_Maximum2DDiameterSlice,E_shape_Maximum3DDiameter,E_shape_Sphericity,E_shape_SurfaceArea,E_shape_SurfaceVolumeRatio
0,0.899454,0.224815,0.001447,0.326354,0.134208,0.040864,0.068070,0.926275,0.119868,0.930081,...,0.424989,0.584200,0.346922,0.710680,0.615759,0.643258,0.429929,0.245488,0.319106,0.351956
1,0.922878,0.132621,0.000116,0.271645,0.068688,0.319848,0.115886,0.955338,0.085204,0.977984,...,0.157530,0.080705,0.043912,0.127181,0.109208,0.117053,0.037356,0.562166,0.030502,0.416652
2,0.997968,0.064852,0.000021,0.078361,0.006171,0.497102,0.000000,0.994021,0.000000,0.992534,...,0.188586,0.116256,0.199095,0.071409,0.173198,0.186908,0.185251,0.499569,0.067769,0.372533
3,0.994541,0.113182,0.000735,0.129266,0.013172,0.793355,0.057687,1.000000,0.013530,0.998967,...,0.507517,0.286732,0.419537,0.330195,0.400249,0.368184,0.592538,0.813031,0.223757,0.038046
4,0.807359,0.621585,0.038355,0.561155,0.168886,0.391687,0.329135,0.944827,0.272838,0.985237,...,0.511882,0.171372,0.372050,0.246110,0.238267,0.231239,0.425582,0.897422,0.146591,0.052481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.849404,0.341635,0.017056,0.460175,0.186524,0.167126,0.208101,0.909394,0.191784,0.930337,...,0.631291,0.484783,0.574650,0.593614,0.488667,0.533749,0.735848,0.390165,0.511117,0.125770
115,0.868489,0.240962,0.013576,0.404084,0.151368,0.144067,0.159340,0.903973,0.154817,0.917618,...,0.667600,0.516476,0.585667,0.591313,0.486231,0.584679,0.629968,0.518820,0.459561,0.069678
116,0.682934,0.464526,0.006866,0.669243,0.371421,0.181383,0.274059,0.749091,0.382596,0.791379,...,0.572761,0.262346,0.501283,0.350160,0.314371,0.314106,0.539552,0.603181,0.268111,0.097392
117,0.710082,0.395290,0.015112,0.610496,0.303111,0.144862,0.226339,0.800391,0.327451,0.856753,...,0.621492,0.297452,0.577031,0.317846,0.382874,0.351769,0.619808,0.777892,0.256521,0.035598


In [6]:
# Compute the correlation matrix
corr_matrix = df_normalized.corr().abs()

corr_matrix_dimensions = corr_matrix.shape

print("Dimensions of the correlation matrix:", corr_matrix_dimensions)

# Set the threshold for considering correlation as high
threshold = 0.9

# Find pairs of highly correlated features
high_corr_pairs = np.where(corr_matrix > threshold)
high_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_pairs) if x != y and x < y]


Dimensions of the correlation matrix: (428, 428)


In [7]:
feature1 = 'A_LOG_1.0S_FO_IQR'  
feature2 = 'A_LOG_1.0S_FO_Mean'  

correlation_value = corr_matrix.loc[feature1, feature2]
print(f"Correlation between {feature1} and {feature2}: {correlation_value}")

Correlation between A_LOG_1.0S_FO_IQR and A_LOG_1.0S_FO_Mean: 0.9472574553577717


In [8]:
feature1 = 'A_LOG_1.0S_FO_IQR'  
feature2 = 'A_LOG_1.0S_FO_MAD'  

correlation_value = corr_matrix.loc[feature1, feature2]
print(f"Correlation between {feature1} and {feature2}: {correlation_value}")

Correlation between A_LOG_1.0S_FO_IQR and A_LOG_1.0S_FO_MAD: 0.9936202031210531


In [9]:
df_target = df[['Event', 'Time']]

In [10]:
# Adding ''A_LOG_1.0S_FO_IQR', 'A_LOG_1.0S_FO_Kurtosis' to the first subset

In [10]:
df_new = df_normalized[['Age', 'KPS', 'A_LOG_1.0S_FO_10Percentile', 'A_LOG_1.0S_FO_90Percentile', 'A_LOG_1.0S_FO_Energy', 'A_LOG_1.0S_FO_Entropy', 'A_LOG_1.0S_FO_IQR', 'A_LOG_1.0S_FO_Kurtosis' ]]
df_new 

,Age,KPS,A_LOG_1.0S_FO_10Percentile,A_LOG_1.0S_FO_90Percentile,A_LOG_1.0S_FO_Energy,A_LOG_1.0S_FO_Entropy,A_LOG_1.0S_FO_IQR,A_LOG_1.0S_FO_Kurtosis
0,0.533333,1.000000,0.899454,0.224815,0.001447,0.326354,0.134208,0.040864
1,0.633333,0.666667,0.922878,0.132621,0.000116,0.271645,0.068688,0.319848
2,0.716667,0.666667,0.997968,0.064852,0.000021,0.078361,0.006171,0.497102
3,0.000000,0.666667,0.994541,0.113182,0.000735,0.129266,0.013172,0.793355
4,0.250000,1.000000,0.807359,0.621585,0.038355,0.561155,0.168886,0.391687
...,...,...,...,...,...,...,...,...
114,0.800000,0.333333,0.849404,0.341635,0.017056,0.460175,0.186524,0.167126
115,0.933333,0.666667,0.868489,0.240962,0.013576,0.404084,0.151368,0.144067
116,0.666667,0.666667,0.682934,0.464526,0.006866,0.669243,0.371421,0.181383
117,0.433333,0.666667,0.710082,0.395290,0.015112,0.610496,0.303111,0.144862


In [11]:

df_combined = pd.concat([df_new,df_target], axis = 1)
print(df_combined )
X, event, y = df_combined.iloc[:,:-2].values, df.iloc[:,-2].values.astype(int), df.iloc[:,-1].values
h = df_combined.columns[:-2]
X = pd.DataFrame(X, columns=h)
event = pd.DataFrame(event)
y = pd.DataFrame(y)
_, y_sksurv = get_x_y(df_combined, df_combined.columns[-2:], 1)

# split train and test set
X_train, X_test, event_train, event_test, y_train, y_test, y_sksurv_train, y_sksurv_test = train_test_split(X, event, y, y_sksurv, test_size=0.2, random_state=2024)
times_train = np.unique(y_train.values.reshape(-1))
times_test = np.unique(y_test.values.reshape(-1))

# compute reference lower bounds
ref_model = RandomSurvivalForest(n_estimators=100, max_depth=3, random_state=2024)
ref_model.fit(X_train, y_sksurv_train)
ref_S_hat = ref_model.predict_survival_function(X_train)
ref_estimates = np.array([f(times_train) for f in ref_S_hat])
ibs_loss_per_sample = compute_ibs_per_sample(event_train, y_train, event_train, y_train, ref_estimates, times_train)

labelsdir = pathlib.Path('/tmp/warm_lb_labels')
labelsdir.mkdir(exist_ok=True, parents=True)

labelpath = labelsdir / 'warm_label.tmp'
labelpath = str(labelpath)

pd.DataFrame(ibs_loss_per_sample, columns=['class_labels']).to_csv(labelpath, header='class_labels', index=None)

# fit model
config = {
    "look_ahead": True,
    "diagnostics": True,
    "verbose": False,

    "regularization": 0.01,
    "uncertainty_tolerance": 0.0,
    "upperbound": 0.0,
    "depth_budget": 5,
    "minimum_captured_points": 7,

    "model_limit": 100,

    "warm_LB": True,
    "path_to_labels": labelpath,
  }



model = OSST(config)
model.fit(X_train, event_train, y_train)
    
# evaluation
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.time
print(f"Model finished execution with {df_new.columns}")
print("# of leaves: {}".format(n_leaves))


: 

In [13]:
# Further adding 'A_LOG_1.0S_FO_Kurtosis' to the first subset

In [14]:
df_new = df_normalized[['Age', 'KPS', 'A_LOG_1.0S_FO_10Percentile', 'A_LOG_1.0S_FO_90Percentile', 'A_LOG_1.0S_FO_Energy', 'A_LOG_1.0S_FO_Entropy' , 'A_LOG_1.0S_FO_IQR', 'A_LOG_1.0S_FO_Kurtosis', 'A_LOG_1.0S_FO_Maximum', 'A_LOG_1.0S_FO_Mean', 'A_LOG_1.0S_FO_MAD', 'A_LOG_1.0S_FO_Median' ]]
df_new 

,Age,KPS,A_LOG_1.0S_FO_10Percentile,A_LOG_1.0S_FO_90Percentile,A_LOG_1.0S_FO_Energy,A_LOG_1.0S_FO_Entropy,A_LOG_1.0S_FO_IQR,A_LOG_1.0S_FO_Kurtosis,A_LOG_1.0S_FO_Maximum,A_LOG_1.0S_FO_Mean,A_LOG_1.0S_FO_MAD,A_LOG_1.0S_FO_Median
0,0.533333,1.000000,0.899454,0.224815,0.001447,0.326354,0.134208,0.040864,0.068070,0.926275,0.119868,0.930081
1,0.633333,0.666667,0.922878,0.132621,0.000116,0.271645,0.068688,0.319848,0.115886,0.955338,0.085204,0.977984
2,0.716667,0.666667,0.997968,0.064852,0.000021,0.078361,0.006171,0.497102,0.000000,0.994021,0.000000,0.992534
3,0.000000,0.666667,0.994541,0.113182,0.000735,0.129266,0.013172,0.793355,0.057687,1.000000,0.013530,0.998967
4,0.250000,1.000000,0.807359,0.621585,0.038355,0.561155,0.168886,0.391687,0.329135,0.944827,0.272838,0.985237
...,...,...,...,...,...,...,...,...,...,...,...,...
114,0.800000,0.333333,0.849404,0.341635,0.017056,0.460175,0.186524,0.167126,0.208101,0.909394,0.191784,0.930337
115,0.933333,0.666667,0.868489,0.240962,0.013576,0.404084,0.151368,0.144067,0.159340,0.903973,0.154817,0.917618
116,0.666667,0.666667,0.682934,0.464526,0.006866,0.669243,0.371421,0.181383,0.274059,0.749091,0.382596,0.791379
117,0.433333,0.666667,0.710082,0.395290,0.015112,0.610496,0.303111,0.144862,0.226339,0.800391,0.327451,0.856753


In [15]:

    df_combined = pd.concat([df_new,df_target], axis = 1)
    print(df_combined )
    X, event, y = df_combined.iloc[:,:-2].values, df.iloc[:,-2].values.astype(int), df.iloc[:,-1].values
    h = df_combined.columns[:-2]
    X = pd.DataFrame(X, columns=h)
    event = pd.DataFrame(event)
    y = pd.DataFrame(y)
    _, y_sksurv = get_x_y(df_combined, df_combined.columns[-2:], 1)
    
    # split train and test set
    X_train, X_test, event_train, event_test, y_train, y_test, y_sksurv_train, y_sksurv_test = train_test_split(X, event, y, y_sksurv, test_size=0.2, random_state=2024)
    times_train = np.unique(y_train.values.reshape(-1))
    times_test = np.unique(y_test.values.reshape(-1))

    # compute reference lower bounds
    ref_model = RandomSurvivalForest(n_estimators=100, max_depth=3, random_state=2024)
    ref_model.fit(X_train, y_sksurv_train)
    ref_S_hat = ref_model.predict_survival_function(X_train)
    ref_estimates = np.array([f(times_train) for f in ref_S_hat])
    ibs_loss_per_sample = compute_ibs_per_sample(event_train, y_train, event_train, y_train, ref_estimates, times_train)

    labelsdir = pathlib.Path('/tmp/warm_lb_labels')
    labelsdir.mkdir(exist_ok=True, parents=True)

    labelpath = labelsdir / 'warm_label.tmp'
    labelpath = str(labelpath)

    pd.DataFrame(ibs_loss_per_sample, columns=['class_labels']).to_csv(labelpath, header='class_labels', index=None)

    # fit model
    config = {
        "look_ahead": True,
        "diagnostics": True,
        "verbose": False,

        "regularization": 0.01,
        "uncertainty_tolerance": 0.0,
        "upperbound": 0.0,
        "depth_budget": 5,
        "minimum_captured_points": 7,

        "model_limit": 100,
    
        "warm_LB": True,
        "path_to_labels": labelpath,
      }


    
    model = OSST(config)
    model.fit(X_train, event_train, y_train)
        
    # evaluation
    n_leaves = model.leaves()
    n_nodes = model.nodes()
    time = model.time
    print(f"Model finished execution with {df_new.columns}")
    

          Age       KPS  A_LOG_1.0S_FO_10Percentile  \
0    0.533333  1.000000                    0.899454   
1    0.633333  0.666667                    0.922878   
2    0.716667  0.666667                    0.997968   
3    0.000000  0.666667                    0.994541   
4    0.250000  1.000000                    0.807359   
..        ...       ...                         ...   
114  0.800000  0.333333                    0.849404   
115  0.933333  0.666667                    0.868489   
116  0.666667  0.666667                    0.682934   
117  0.433333  0.666667                    0.710082   
118  0.516667  0.666667                    0.737363   

     A_LOG_1.0S_FO_90Percentile  A_LOG_1.0S_FO_Energy  A_LOG_1.0S_FO_Entropy  \
0                      0.224815              0.001447               0.326354   
1                      0.132621              0.000116               0.271645   
2                      0.064852              0.000021               0.078361   
3                  